Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label Pathology App - HoVerNet-Nuclei Classification and Segmentation

***The pathology cell classification and segmentation analysis with MONAI Label***

In this notebook, we show the application of cell classification and segmentation at the same time with MONAI Label. The nuclei classification and segmentation is predicted via HoVerNet. This tutorial uses QuPath as the client viewer, and we demonstrate the end-to-end workflow with MONAI Label + HoVerNet integration.

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet1.png)

## 1. Prepare MONAI Label

## Setup environment

### Prerequisites
- **Install MONAI Label** weekly preview release: 

In [ ]:
!pip install monailabel

### 1.1 Download pathology app
Choose "pathology" as the app

In [ ]:
!monailabel apps --download --name pathology --output apps

### 1.2 Download sample data
Download the sample data from digital slide archive with wget and save to the **datasets** folder for further use.

In [ ]:
!mkdir datasets
!wget "https://demo.kitware.com/histomicstk/api/v1/item/5d5c07509114c049342b66f8/download" -O "datasets/JP2K-33003-1.svs"

HoVerNet model is trained with CoNSeP datasets, the original dataset can be downloaded from <https://warwick.ac.uk/fac/cross_fac/tia/data/HoVerNet/>.

### 1.3 Starting MONAI Label Server

The command will start MONAI Label server with the pre-trained HoVerNet model. 

Such as

```bash
monailabel start_server --app <path to pathology app> --studies <path to downloaded datafolder> --conf models hovernet_nuclei
```
In the command, ```--app``` specifies the pathology app path, ```--studies``` specifies the dataset folder path.

In [ ]:
!monailabel start_server --app apps/pathology --studies datasets/ --conf models hovernet_nuclei

## 2. Install QuPath and MONAI Label Plugin

[QuPath](https://qupath.github.io/) is an open software for bioimage analysis. it can be used as an efficient tool for annotation and visualization of whole slide images. 

Note: Nvidia does not maintain the thrid-party visualization tools. Please refer to [installation guide](https://qupath.readthedocs.io/en/stable/docs/intro/installation.html) for details or trouble shooting. 

In this tutorial, we provide minimum steps for QuPath setup. 

**Step 1**: Get QuPath from website.  

Go to the [QuPath website](https://qupath.github.io/), click "Download for Linux" to pull the *tar.xz* file

Extract the QuPath software by:

```bash
# E.g., QuPath 0.3.2
tar -xf <QuPath-0.3.2-Linux.tar.xz>
```

**Step 2**: Start QuPath with excutable launcher file.

Start QuPath by opening a **Terminal**, Use ```chmod u+x /path/to/QuPath/bin/QuPath``` to make the extracted QuPath launcher executable.
If you suffer from problem, please refer to QuPath [QA](https://github.com/qupath/qupath/releases/tag/v0.3.2).

Then type or drag the launcher file: ```/path/to/QuPath/bin/QuPath``` to the terminal, and enter to open QuPath UI.

**Step 3** Install MONAI Label Plugin for QuPath

Download [qupath-extension-monailabel-0.3.1.jar](https://github.com/Project-MONAI/MONAILabel/releases/download/data/qupath-extension-monailabel-0.3.1.jar)
and **drag the jar** on top of the running QuPath application window (black screen area) to install the extension.
If you have previously installed then make sure to **_remove/uninstall_** the extension before updating.

<p align = "center"><img src="./figures/monailabel_pathology_qupath/qupath_plugin.png" alt="drawing" width="700"/></p>
<p align = "center">Drag the downloaded MONAI Label QuPath plugin JAR file into QuPath viewer UI.</p>

Note: Make sure MONAILabel Server URL (Host+Port) is correct through `Preferences`.



## 3. Nuclei Classification and Segmentation with Interactions

Below is the instruction of using HoVerNet for labeling nucleis given WSI or other digital pathology images using QuPath. Pre-trained model is provided and loaded by MONAI Label server.


### 3.1 Start QuPath, MONAI Label Plugin, and Fetch Next Image

- On the menu bar, navigate and click MONAI Label. Click *Next Sample/Patch* to load data specified in the start_server command. If the QuPath starts from beginning, the button will automatically load next whole slide image.

<p align = "center"><img src="./figures/monailabel_pathology_qupath/qupath2.png" alt="drawing" width="800"/></p>

 - Users can also use the **Next Sample/Patch** button to select a patch ROI automatically, by selecting **Next Patch**. （Uncheck the **Next Patch** box will load next whole slide image.）

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet2.png)

### 3.2 Select ROI by Rectangle Tool

First, let's show patch inference with the rectangle ROI tool.

- Users can automatically select the rectangle ROI by fetching "Next Patch", or use the QuPath rectangle tool to pick an ROI.

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet3.png)

### 3.3 Run Nuclei Classication and Segmentation Simultaneously

- After selecting the rectangle ROI, users can navigate to the MONAI Label plugin and click **Annotations**.
Then select models in the pop-up interface, and click **OK** to run inference. 

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet4.png)

MONAI Label server will run auto segmentation with the selected patch data and return the segmentation with polygons.

### 3.4 Submit ROI Label for Training

- In the next step, users can edit the predicted nuclei segmentation. Once the annotation is done, users can navigate to MONAI Label plugin and click **Submit Label** to save the ground truth.

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet6.png)


### 3.5 Run Interactive ROI-based Segmentation

- Users can also use the interactive ROI tool to get auto-segmentation with one ROI draw:
- Draw any rectangle ROI, and MONAI Label server will return segmentation immediately:

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet7.png)

## 4. Training Nuclei Segmentation

Once users finish annotations on several patch images, MONAI Label provide training option to fine-tune the model.

- Navigate to MONAI Label plugin and click **Training**

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet9.png)

- In the training option pop-up interface, select **hovernet_nuclei**, select training customized training options. Uncheck **pretrained** to start training model from scratch. Finally, click **OK** to start training in MONAI Label server.

![hovernet1](./figures/monailabel_pathology_HoVerNet_QuPath/hovernet10.png)



## 5. Repeat the Interactive Labeling Process 

Repeated fetch data and interactive learning process to section 3.1, until a satisfied model is achieved or entire datasets/WSIs are annotated. 


## 6. Conclusion

This tutorial demonstrates the basic usage of HoVerNet, nuclei classification and segmentation at the same time with MONAI Label. We introduced several inference strategies including ROI tool, interactive ROI tool. We show how to submit annotated labels, then the steps of training are demonstrated.

In the MONAI Label HoVerNet integration, we highlight the interactive labeling experience with classification and segmentation at the same time from a whole slide image. MONAI Label achieves annotation actively and gradually improves the model's classification and segmentation performance during labeling. 
